## First, import relevant libraries 

In [1]:
import numpy as np
import pandas as pd

## Loading and merging the movie data 

In [2]:
movie_data=pd.read_csv('data/ratings.csv')
#movie_data.head(10)

In [3]:
movies=pd.read_csv('data/movies.csv')
#movies.head(10)

In [4]:
movie_data=movie_data.merge(movies,on='movieId',how='left')
#movie_data.head(10)

In [5]:
#get average rating from movies

rating = pd.DataFrame(movie_data.groupby('title')['rating'].mean().sort_values(ascending=False))
#rating.head(10)

In [6]:
#find out the number of ratings for each movie

rating['total_rated']=pd.DataFrame(movie_data.groupby('title')['rating'].count())
rating.sort_values(by="total_rated", ascending=False)

,rating,total_rated
title,,
Forrest Gump (1994),4.164134,329
"Shawshank Redemption, The (1994)",4.429022,317
Pulp Fiction (1994),4.197068,307
"Silence of the Lambs, The (1991)",4.161290,279
"Matrix, The (1999)",4.192446,278
...,...,...
Sebastian Maniscalco: What's Wrong with People? (2012),3.500000,1
Fat Man and Little Boy (1989),3.500000,1
Hotel Chevalier (Part 1 of 'The Darjeeling Limited') (2007),3.500000,1


In [7]:
good_movies = rating[(rating.rating >=4) & (rating.total_rated >= 100)].sort_values(by="rating", ascending=False)
#good_movies

## Create Pivot table to find out the correlations between the movies

In [8]:
movie_user=movie_data.pivot_table(index='userId',columns='title',values='rating')
movie_user.head(10)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


#test recommender system with any movie title

correlation=movie_user.corrwith(movie_user["Titanic (1997)"])
#correlation.head(10)

#remove empty values and join total_rated column
recommandation=pd.DataFrame(correlation,columns=['correlation'])
recommandation.dropna(inplace=True)
recommandation=recommandation.join(rating['total_rated'])
#recommandation.head()

#only select movies that are rated at least 100 times + merge movies genres

recc=recommandation[recommandation['total_rated']>100].sort_values('correlation',ascending=False).reset_index()
recc=recc.merge(movies,on='title',how='left')
recc.head(10)

## Get 5 good  rated films from a user

In [9]:
get_best_rated_movies = int(input("Pls type in your User ID:"))

individual_rating = movie_data[(movie_data.userId == get_best_rated_movies) & (movie_data.rating >= 4.5)].sort_values(by="rating", ascending=False)
#print(individual_rating)
chose_user_movie = individual_rating.iloc[:].sample(5)[["title", "rating"]].sort_values(by="rating", ascending=False)
print(chose_user_movie)


Pls type in your User ID:5
                                      title  rating
557                        Pinocchio (1940)     5.0
546        In the Name of the Father (1993)     5.0
555  Snow White and the Seven Dwarfs (1937)     5.0
527               Heavenly Creatures (1994)     5.0
556             Beauty and the Beast (1991)     5.0


#for i in chose_user_movie[0]["title"]:
test = chose_user_movie["title"].iloc[0]
print(test)

## Find out the best correlated movies with the the above movies, merge them 

In [10]:
count=0

for i in chose_user_movie["title"]:
    correlation=movie_user.corrwith(movie_user[i])
    #print(correlation)#.head(10)
    recommandation=pd.DataFrame(correlation,columns=['correlation'])
    recommandation.dropna(inplace=True)
    recommandation=recommandation.join(rating['total_rated'])
    #recommandation.head()
    recc=recommandation[recommandation['total_rated']>100].sort_values('correlation',ascending=False).reset_index()
    recc=recc.merge(movies,on='title',how='left')
    df = pd.DataFrame(data=recc)
    
    if count==0:
        count+=1 
        df_concat = df
    else:
        df_concat=pd.concat([df_concat, df])

#print(df_concat)
    
    #print(df.iloc[0])
    
#for i in recc:
#    print(i)
    
   # df = pd.DataFrame(data=recc) 

   


C:\Users\Daniel Schmitz\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Daniel Schmitz\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


## Filter data by removing all movies which have less than 0.5 correlation

In [11]:
df_concat = df_concat[df_concat["correlation"]>=0.5]

In [12]:
#df_concat.head(5)

In [13]:
user_history=list(movie_data.loc[movie_data['userId'] == get_best_rated_movies]["movieId"]) 
#print(user_history)


In [14]:
user_history2=list(df_concat["movieId"]) 
#print(user_history2)


## Create a list of movie recommendation. Also filter out movies which the user already have seen

In [15]:
recommandation_list=[]
for movie in user_history2:
    if movie not in user_history:
        if movie not in recommandation_list:
            recommandation_list.append(movie)
            
    

In [16]:
#print(recommandation_list)

In [17]:
#movies.head(5)

## Draw a sample(3) out of list and give final recommendations

In [18]:
get_recommendation = movies.set_index("movieId").loc[recommandation_list]
get_recommendation.sample(3)

,title,genres
movieId,,
4963,Ocean's Eleven (2001),Crime|Thriller
1036,Die Hard (1988),Action|Crime|Thriller
1222,Full Metal Jacket (1987),Drama|War


In [19]:
#movie_data.loc[movie_data['userId'] == 2]